In [ ]:
def radial_stress(r,theta,data):
    zeta = data['R']/r
    aniso = data['smin']/data['smax']
    return data['smax']/2. * ( (1.+aniso)*(1.-zeta**2.) - 
                           (1.-aniso)*(1.-4.*zeta**2. + 3.*zeta**4.)*np.cos(2.*theta)) + data['p']*zeta**2

In [ ]:
def circum_stress(r,theta,data):
    zeta = data['R']/r
    aniso = data['smin']/data['smax']
    return data['smax']/2. * ( (1.+aniso)*(1.+zeta**2.) + 
                           (1.-aniso)*(1. + 3.*zeta**4.)*np.cos(2.*theta)) -  data['p']*zeta**2

In [ ]:
def shear_stress(r,theta,data):
    zeta = data['R']/r
    aniso = data['smin']/data['smax']
    return data['smax']/2. * ( (1.-aniso)*(1. + 2.*zeta**2. - 3.*zeta**4.)*np.sin(2.*theta))

In [ ]:
def line_plot_stress_solution():
    rad = np.linspace(input_data['R'],5.*input_data['R'],1000)
    plt.rcParams['figure.figsize'] = (15, 5)

    fig,ax = plt.subplots(ncols=3)
    ax[0].plot(rad,radial_stress(rad,0.,input_data)/1e6,label=r'$\sigma_{rr}(r,\theta = 0)$')
    ax[0].plot(rad,circum_stress(rad,0.,input_data)/1e6,label=r'$\sigma_{\theta\theta}(r,\theta = 0)$')
    ax[0].plot(rad,shear_stress(rad,0.,input_data)/1e6,label=r'$\sigma_{r\theta}(r,\theta = 0)$')

    ax[1].plot(rad,radial_stress(rad,np.pi/4.,input_data)/1e6,label=r'$\sigma_{rr}(r,\theta = \pi/4)$')
    ax[1].plot(rad,circum_stress(rad,np.pi/4.,input_data)/1e6,label=r'$\sigma_{\theta\theta}(r,\theta = \pi/4)$')
    ax[1].plot(rad,shear_stress(rad,np.pi/4.,input_data)/1e6,label=r'$\sigma_{r\theta}(r,\theta = \pi/4)$')

    ax[2].plot(rad,radial_stress(rad,np.pi/2.,input_data)/1e6,label=r'$\sigma_{rr}(r,\theta = \pi/2)$')
    ax[2].plot(rad,circum_stress(rad,np.pi/2.,input_data)/1e6,label=r'$\sigma_{\theta\theta}(r,\theta = \pi/2)$')
    ax[2].plot(rad,shear_stress(rad,np.pi/2.,input_data)/1e6,label=r'$\sigma_{r\theta}(r,\theta = \pi/2)$')

    for i in range(3):
        ax[i].legend()
        ax[i].set_xlabel(r'$r$ / m')
        ax[i].set_ylabel(r'$\sigma$ / MPa')
        #ax[i].set_ylim(top=maximum_stress(input_data))
    #fig.suptitle(r'Spannungsverteilung um leeres Bohrloch in 479 m Teufe',size=18,weight='bold')
    fig.tight_layout();
    #plt.subplots_adjust(top=0.88)
    #fig.savefig('Kirchhoff_stress_distribution_empty.pdf')

In [ ]:
cart_to_cyl = lambda x,y: [np.sqrt(x**2+y**2), (np.arctan(y/x))]

In [ ]:
def vector_to_tensor(vec):
    return np.array([[vec[0],vec[2]],
                    [vec[2],vec[1]]])

In [ ]:
def calc_principal_stresses(s):
    eigenValues, eigenVectors = np.linalg.eig(s)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return eigenValues, eigenVectors

In [ ]:
def plot_stresses_polar():
    t_rad = input_data['R']
    X, Y = np.meshgrid(np.linspace(0.1,5*t_rad,1000),np.linspace(0.1,5*t_rad,1000))
    #cylindical coordinates from Cartesian grid
    r, theta = cart_to_cyl(X,Y)
    
    s_rr = radial_stress(r,theta,input_data)
    s_tt = circum_stress(r,theta,input_data)
    s_rt = shear_stress(r,theta,input_data)
    
    s_rr_init = radial_stress(r+t_rad*1000,theta,input_data)
    s_tt_init = circum_stress(r+t_rad*1000,theta,input_data)
    s_rt_init = shear_stress(r+t_rad*1000,theta,input_data)

    fig, ax = plt.subplots(ncols=3, nrows=3,figsize=(18,18))
    #
    l1=ax[0][0].contourf(X,Y,np.ma.masked_array(s_rr_init/1e6))
    l2=ax[0][1].contourf(X,Y,np.ma.masked_array(s_tt_init/1e6))
    l3=ax[0][2].contourf(X,Y,np.ma.masked_array(s_rt_init/1e6))
    fig.colorbar(l1,ax=ax[0][0],format='%.2f')
    fig.colorbar(l2,ax=ax[0][1],format='%.2f')
    fig.colorbar(l3,ax=ax[0][2],format='%.2f')
    #
    l1=ax[1][0].contourf(X,Y,np.ma.masked_array(s_rr/1e6,mask=r<input_data['R']))
    l2=ax[1][1].contourf(X,Y,np.ma.masked_array(s_tt/1e6,mask=r<input_data['R']))
    l3=ax[1][2].contourf(X,Y,np.ma.masked_array(s_rt/1e6,mask=r<input_data['R']))
    fig.colorbar(l1,ax=ax[1][0],format='%.2f')
    fig.colorbar(l2,ax=ax[1][1],format='%.2f')
    fig.colorbar(l3,ax=ax[1][2],format='%.2f')
    #
    l1=ax[2][0].contourf(X,Y,np.ma.masked_array((s_rr - s_rr_init)/1e6,mask=r<input_data['R']))
    l2=ax[2][1].contourf(X,Y,np.ma.masked_array((s_tt - s_tt_init)/1e6,mask=r<input_data['R']))
    l3=ax[2][2].contourf(X,Y,np.ma.masked_array((s_rt - s_rt_init)/1e6,mask=r<input_data['R']))
    fig.colorbar(l1,ax=ax[2][0],format='%.2f')
    fig.colorbar(l2,ax=ax[2][1],format='%.2f')
    fig.colorbar(l3,ax=ax[2][2],format='%.2f')
    for i in range(3):
        for j in range(3):
            ax[i][j].set_aspect('equal')
            ax[i][j].set_xlabel(r'$x$ / m')
            ax[i][j].set_ylabel(r'$y$ / m')
    ax[0][0].set_title(r'$\sigma_{rr}^0$ / MPa')
    ax[0][1].set_title(r'$\sigma_{\theta\theta}^0$ / MPa')
    ax[0][2].set_title(r'$\sigma_{r\theta}^0$ / MPa')
    
    ax[1][0].set_title(r'$\sigma_{rr}$ / MPa')
    ax[1][1].set_title(r'$\sigma_{\theta\theta}$ / MPa')
    ax[1][2].set_title(r'$\sigma_{r\theta}$ / MPa')
    
    ax[2][0].set_title(r'$\Delta\sigma_{rr}$ / MPa')
    ax[2][1].set_title(r'$\Delta\sigma_{\theta\theta}$ / MPa')
    ax[2][2].set_title(r'$\Delta\sigma_{r\theta}$ / MPa')
    fig.tight_layout();

In [ ]:
def Hoek_Brown(s3):
    A = input_data['A']
    B = input_data['B']
    sqrt = A*s3 + B**2
    arg = np.sqrt(sqrt*np.heaviside(sqrt,0))
    return arg*np.heaviside(arg,0)

In [ ]:
def forloop(a,b):
    results = {
        's1_init': np.zeros((b-a,len(X))),
        's3_init': np.zeros((b-a,len(X))),
        's1vx_init': np.zeros((b-a,len(X))),
        's1vy_init': np.zeros((b-a,len(X))),
        's3vx_init': np.zeros((b-a,len(X))),
        's3vy_init': np.zeros((b-a,len(X))),
        #
        's1_excav': np.zeros((b-a,len(X))),
        's3_excav': np.zeros((b-a,len(X))),
        's1vx_excav': np.zeros((b-a,len(X))),
        's1vy_excav': np.zeros((b-a,len(X))),
        's3vx_excav': np.zeros((b-a,len(X))),
        's3vy_excav': np.zeros((b-a,len(X))),
        #
        'ds1': np.zeros((b-a,len(X))),
        'ds3': np.zeros((b-a,len(X))),
        'ds1vx': np.zeros((b-a,len(X))),
        'ds1vy': np.zeros((b-a,len(X))),
        'ds3vx': np.zeros((b-a,len(X))),
        'ds3vy': np.zeros((b-a,len(X))),
        #
        'UL': np.zeros((b-a,len(X))),
        'HB': np.zeros((b-a,len(X)))
    }
    
    for i in range(a,b):
        for j in range(len(X)):
            R = np.array([[np.cos(theta[i,j]),np.sin(theta[i,j])],[-np.sin(theta[i,j]),np.cos(theta[i,j])]])
            sigma_vec = np.array([s_rr_init[i,j],s_tt_init[i,j],s_rt_init[i,j]])
            sigma_xy = R.T.dot((vector_to_tensor(sigma_vec)).dot(R))
            e_val, e_vec = calc_principal_stresses(sigma_xy)
            results['s1_init'][i-a,j] = e_val[0]
            results['s3_init'][i-a,j] = e_val[1]
            results['s1vx_init'][i-a,j] = e_vec[0][0]
            results['s1vy_init'][i-a,j] = e_vec[1][0]
            results['s3vx_init'][i-a,j] = e_vec[0][1]
            results['s3vy_init'][i-a,j] = e_vec[1][1]
            #
            sigma_vec = np.array([s_rr_excav[i,j],s_tt_excav[i,j],s_rt_excav[i,j]])
            sigma_xy = R.T.dot((vector_to_tensor(sigma_vec)).dot(R))
            e_val, e_vec = calc_principal_stresses(sigma_xy)
            results['s1_excav'][i-a,j] = e_val[0]
            results['s3_excav'][i-a,j] = e_val[1]
            results['s1vx_excav'][i-a,j] = e_vec[0][0]
            results['s1vy_excav'][i-a,j] = e_vec[1][0]
            results['s3vx_excav'][i-a,j] = e_vec[0][1]
            results['s3vy_excav'][i-a,j] = e_vec[1][1]
            #
            results['HB'][i-a,j] = np.heaviside(-(Hoek_Brown(results['s3_excav'][i-a,j]) - (results['s1_excav'][i-a,j]-results['s3_excav'][i-a,j])),0)
            #
            sigma_vec = np.array([s_rr_excav[i,j]-s_rr_init[i,j],s_tt_excav[i,j]-s_tt_init[i,j],s_rt_excav[i,j]-s_rt_init[i,j]])
            sigma_xy = R.T.dot((vector_to_tensor(sigma_vec)).dot(R))
            e_val, e_vec = calc_principal_stresses(sigma_xy)
            results['ds1'][i-a,j] = e_val[0]
            results['ds3'][i-a,j] = e_val[1]
            results['ds1vx'][i-a,j] = e_vec[0][0]
            results['ds1vy'][i-a,j] = e_vec[1][0]
            results['ds3vx'][i-a,j] = e_vec[0][1]
            results['ds3vy'][i-a,j] = e_vec[1][1]
            #
            results['UL'][i-a,j] = np.heaviside(-input_data['sul']-results['ds3'][i-a,j],0)
    return results

In [ ]:
def plot_stresses_principal():    
    fig, ax = plt.subplots(ncols=3, nrows=3,figsize=(18,18))
    #
    l1=ax[0][0].contourf(X,Y,np.ma.masked_array(result['s1_init']/1e6))
    strm = ax[0][0].streamplot(X[0],X[0],result['s1vx_init'],result['s1vy_init'],density=1, linewidth=2, color='black')
    l2=ax[0][1].contourf(X,Y,np.ma.masked_array(result['s3_init']/1e6))
    strm = ax[0][1].streamplot(X[0],X[0],result['s3vx_init'],result['s3vy_init'],density=1, linewidth=2, color='black')
    l3=ax[0][2].contourf(X,Y,np.ma.masked_array((result['s1_init']-result['s3_init'])/2e6))
    fig.colorbar(l1,ax=ax[0][0],format='%.2f')
    fig.colorbar(l2,ax=ax[0][1],format='%.2f')
    fig.colorbar(l3,ax=ax[0][2],format='%.2f')
    #
    l1=ax[1][0].contourf(X,Y,np.ma.masked_array(result['s1_excav']/1e6,mask=r<input_data['R']))
    strm = ax[1][0].streamplot(X[0],X[0],np.ma.masked_array(result['s1vx_excav'],mask=r<input_data['R']),np.ma.masked_array(result['s1vy_excav'],mask=r<input_data['R']),density=1, linewidth=2, color='black')
    l2=ax[1][1].contourf(X,Y,np.ma.masked_array(result['s3_excav']/1e6,mask=r<input_data['R']))
    strm = ax[1][1].streamplot(X[0],X[0],np.ma.masked_array(result['s3vx_excav'],mask=r<input_data['R']),np.ma.masked_array(result['s3vy_excav'],mask=r<input_data['R']),density=1, linewidth=2, color='black')
    l3=ax[1][2].contourf(X,Y,np.ma.masked_array((result['s1_excav']-result['s3_excav'])/2e6,mask=r<input_data['R']))
    fig.colorbar(l1,ax=ax[1][0],format='%.2f')
    fig.colorbar(l2,ax=ax[1][1],format='%.2f')
    fig.colorbar(l3,ax=ax[1][2],format='%.2f')
    #
    l1=ax[2][0].contourf(X,Y,np.ma.masked_array((result['ds1'])/1e6,mask=r<input_data['R']))
    strm = ax[2][0].streamplot(X[0],X[0],np.ma.masked_array(result['ds1vx'],mask=r<input_data['R']),np.ma.masked_array(result['ds1vy'],mask=r<input_data['R']),density=1, linewidth=2, color='black')
    l2=ax[2][1].contourf(X,Y,np.ma.masked_array((result['ds3'])/1e6,mask=r<input_data['R']))
    strm = ax[2][1].streamplot(X[0],X[0],np.ma.masked_array(result['ds3vx'],mask=r<input_data['R']),np.ma.masked_array(result['ds3vy'],mask=r<input_data['R']),density=1, linewidth=2, color='black')
    l3=ax[2][2].contourf(X,Y,np.ma.masked_array((result['ds1']-result['ds3'])/2e6,mask=r<input_data['R']))
    fig.colorbar(l1,ax=ax[2][0],format='%.2f')
    fig.colorbar(l2,ax=ax[2][1],format='%.2f')
    fig.colorbar(l3,ax=ax[2][2],format='%.2f')
    for i in range(3):
        for j in range(3):
            ax[i][j].set_aspect('equal')
            ax[i][j].set_xlabel(r'$x$ / m')
            ax[i][j].set_ylabel(r'$y$ / m')
            ax[i][j].set_ylim(0,np.max(X))
            ax[i][j].set_xlim(0,np.max(Y))
    ax[0][0].set_title(r'$\sigma_{1}^0$ / MPa')
    ax[0][1].set_title(r'$\sigma_{3}^0$ / MPa')
    ax[0][2].set_title(r'$\tau_\mathrm{max}^0$ / MPa')
    
    ax[1][0].set_title(r'$\sigma_{1}$ / MPa')
    ax[1][1].set_title(r'$\sigma_{3}$ / MPa')
    ax[1][2].set_title(r'$\tau_\mathrm{max}$ / MPa')
    
    ax[2][0].set_title(r'$\Delta\sigma_{1}$ / MPa')
    ax[2][1].set_title(r'$\Delta\sigma_{3}$ / MPa')
    ax[2][2].set_title(r'$\Delta\tau_\mathrm{max}$ / MPa')
    
    fig.tight_layout();

In [ ]:
def plot_HB():
    A = input_data['A']
    B = input_data['B']
    s3 = np.linspace(-10,50,1000)
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(s3,Hoek_Brown(s3*1e6)/1e6)
    ax.set_xlabel(r'$\sigma_3$ / MPa')
    ax.set_ylabel(r'$(\sigma_1 - \sigma_3)$ / MPa')
    fig.tight_layout();

In [ ]:
def plot_failure():
    
    fig, ax = plt.subplots(ncols=2,figsize=(12,6))
    #
    l1=ax[0].contourf(X,Y,np.ma.masked_array(result['HB'],mask=r<input_data['R']))
    l2=ax[1].contourf(X,Y,np.ma.masked_array(result['UL'],mask=r<input_data['R']))
    #fig.colorbar(l1,ax=ax[0],format='%.2f')
    #fig.colorbar(l2,ax=ax[1],format='%.2f')
    
    for i in range(2):
        ax[i].set_aspect('equal')
        ax[i].set_xlabel(r'$x$ / m')
        ax[i].set_ylabel(r'$y$ / m')
        ax[i].set_ylim(0,np.max(X))
        ax[i].set_xlim(0,np.max(Y))
    ax[0].set_title(r'$H(F_\mathrm{HB})$')
    ax[1].set_title(r'$H(F_\mathrm{ul})$')
    
    fig.tight_layout();

In [ ]:
def plot_anisotropy_variation():
    lambdas = np.linspace(0,1,10)
    input_data['p'] = 0.
    sig_tt_0 = np.zeros_like(lambdas)
    sig_tt_90 = np.zeros_like(lambdas)
    for i,lam in enumerate(lambdas):
        input_data['smin'] = input_data['smax']*lam
        sig_tt_0[i] = circum_stress(input_data['R'], 0, input_data)
        sig_tt_90[i] = circum_stress(input_data['R'], np.pi/2, input_data)

    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(lambdas, sig_tt_0/input_data['smax'],label=r'$\theta = 0$')
    ax.plot(lambdas, sig_tt_90/input_data['smax'],label=r'$\theta = \pi/2$')
    ax.set_xlabel(r'$\lambda = K_0$')
    ax.set_ylabel(r'$\sigma_{\theta\theta}$ / $\sigma_\mathrm{vert}$')
    ax.plot(1,2,marker='o',color='red')
    ax.text(1.02, 1.9, 'isotropic\nstress field',color='red')
    ax.plot(0,3,marker='o',color='red')
    ax.text(0, 2.1, 'uniaxial\nstress concentration\nfactor of circular notch',color='red',rotation=0)
    ax.plot([0,0.33],[-1,0],color='red',ls='--')
    ax.text(0.01,-0.75,'tensile',rotation=20,color='red')
    ax.legend()
    fig.tight_layout();